<p>
 <img src="ayiti_analytics.png" alt="Ayiti Analytics"width=400 />
</p>

# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try How to find a Dance School next to an Elementary school in **Chicago** with a **high score of Instruction** and where **the teachers score** and **the safety score** is also high. These three parameters are very important because it is important for a parent to know that his or her kid is safe at school, that the instruction level is great and the teachers score is good enough.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* Number of School Dance in Chicago
* Number 

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm
from sklearn.cluster import KMeans

import folium

In [2]:
df_capstone = pd.read_csv("Chicago_Public_Schools.csv")

In [3]:
neigh = pd.read_csv('Neighborhoods_2012b.csv')
neig_list = list(set(neigh.PRI_NEIGH))

## Methodology <a name="methodology"></a>

After having done our data processing import our libraries look at the types of each column on jupyter notebook calculate their average describes our numeric columns. our dataset was at about 78 columns after our analysis it was reduced to those columns which are necessary for us:
* NAME_OF_SCHOOL
* Elementary,Middle, or High School
* SAFETY_SCORE
* Instruction Score
* Teachers Score
* Latitude
* Longitude
* COMMUNITY_AREA_NAME
* Location
* Street Address

**We requested data on dance schools located in chicago thanks to foursquare in order to achieve our goal of finding a primary school with a high level of education, high scoring teachers and a high safety score next to a school dance.**
**We have created maps to see the city of Chicago and the addresses of the Schools.**


In [4]:
df_capstone.head()

,School ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street Address,City,State,ZIP Code,Phone Number,Link,Network Manager,...,Freshman on Track Rate %,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NDA,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NDA,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NDA,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NDA,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"


In [5]:
df_capstone.columns

Index(['School ID', 'NAME_OF_SCHOOL', 'Elementary, Middle, or High School',
       'Street Address', 'City', 'State', 'ZIP Code', 'Phone Number', 'Link ',
       'Network Manager', 'Collaborative Name',
       'Adequate Yearly Progress Made? ', 'Track Schedule',
       'CPS Performance Policy Status', 'CPS Performance Policy Level',
       'HEALTHY_SCHOOL_CERTIFIED', 'Safety Icon ', 'SAFETY_SCORE',
       'Family Involvement Icon', 'Family Involvement Score',
       'Environment Icon ', 'Environment Score', 'Instruction Icon ',
       'Instruction Score', 'Leaders Icon ', 'Leaders Score ',
       'Teachers Icon ', 'Teachers Score', 'Parent Engagement Icon ',
       'Parent Engagement Score', 'Parent Environment Icon',
       'Parent Environment Score', 'AVERAGE_STUDENT_ATTENDANCE',
       'Rate of Misconducts (per 100 students) ', 'Average Teacher Attendance',
       'Individualized Education Program Compliance Rate ', 'Pk-2 Literacy %',
       'Pk-2 Math %', 'Gr3-5 Grade Level Math %'

# Analysis <a name="analysis"></a>

In [6]:
df_capstone.mean()

School ID                                  6.096818e+05
ZIP Code                                   6.062973e+04
SAFETY_SCORE                               4.950487e+01
Environment Score                          4.776608e+01
Instruction Score                          4.828850e+01
Rate of Misconducts (per 100 students)     2.108587e+01
ISAT Exceeding Math %                      2.035630e+01
ISAT Exceeding Reading %                   1.566702e+01
ISAT Value Add Math                        6.858974e-02
ISAT Value Add Read                        2.264957e-02
COLLEGE_ENROLLMENT                         6.260530e+02
General Services Route                     3.908481e+01
X_COORDINATE                               1.163016e+06
Y_COORDINATE                               1.885663e+06
Latitude                                   4.184189e+01
Longitude                                 -8.767732e+01
COMMUNITY_AREA_NUMBER                      3.770671e+01
Ward                                       2.206

In [7]:
df_capstone.describe()

,School ID,ZIP Code,SAFETY_SCORE,Environment Score,Instruction Score,Rate of Misconducts (per 100 students),ISAT Exceeding Math %,ISAT Exceeding Reading %,ISAT Value Add Math,ISAT Value Add Read,COLLEGE_ENROLLMENT,General Services Route,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,Ward,Police District
count,566.000000,566.000000,513.000000,513.000000,513.000000,566.000000,476.000000,476.000000,468.000000,468.000000,566.000000,566.000000,5.660000e+02,5.660000e+02,566.000000,566.000000,566.000000,566.000000,566.000000
mean,609681.791519,60629.727915,49.504873,47.766082,48.288499,21.085866,20.356303,15.667017,0.068590,0.022650,626.053004,39.084806,1.163016e+06,1.885663e+06,41.841890,-87.677323,37.706714,22.060071,11.710247
std,8831.060946,20.248024,20.110837,16.215584,17.417176,27.543835,16.850870,15.208361,1.041174,1.042543,448.495105,6.165552,1.605930e+04,3.285688e+04,0.090361,0.058390,21.584135,13.645574,6.688582
min,400018.000000,60605.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-3.500000,-5.000000,21.000000,29.000000,1.118114e+06,1.817242e+06,41.653674,-87.841052,1.000000,1.000000,1.000000
25%,609873.250000,60618.000000,35.000000,37.000000,37.000000,4.525000,9.175000,6.100000,-0.600000,-0.600000,342.250000,34.000000,1.152166e+06,1.860248e+06,41.771866,-87.717152,23.000000,10.000000,7.000000
50%,610052.500000,60625.000000,48.000000,47.000000,47.000000,12.250000,16.000000,10.950000,0.000000,0.000000,504.000000,39.000000,1.163553e+06,1.886889e+06,41.845521,-87.675878,31.500000,21.000000,10.000000
75%,610212.750000,60639.000000,61.000000,58.000000,59.000000,26.975000,24.900000,19.525000,0.700000,0.625000,790.500000,45.000000,1.174256e+06,1.911652e+06,41.913584,-87.636059,58.000000,32.750000,17.000000
max,610544.000000,60827.000000,99.000000,99.000000,99.000000,251.600000,100.000000,100.000000,3.600000,4.900000,4368.000000,49.000000,1.202811e+06,1.950960e+06,42.021064,-87.533007,77.000000,50.000000,25.000000


In [8]:
df_capstone2 = df_capstone [['NAME_OF_SCHOOL','Street Address','Elementary, Middle, or High School','SAFETY_SCORE','Instruction Score','Teachers Score','Latitude','Longitude','COMMUNITY_AREA_NAME','Location']]

In [9]:
df_capstone2_ = df_capstone2[df_capstone2["Elementary, Middle, or High School"] == 'ES']

In [10]:
df_capstone2_ = df_capstone2_[df_capstone2_["SAFETY_SCORE"] >= 70.0]

In [11]:
df_capstone2_= df_capstone2_[df_capstone2_["Instruction Score"] >= 70.0]

In [12]:
df_capstone2_= df_capstone2_[df_capstone2_["Teachers Score"] >= '70.0']

In [13]:
df_capstone2_.describe()

,SAFETY_SCORE,Instruction Score,Latitude,Longitude
count,9.000000,9.000000,9.000000,9.000000
mean,95.222222,88.666667,41.892787,-87.703842
std,7.758508,11.800424,0.098582,0.043923
min,78.000000,71.000000,41.697198,-87.812091
25%,99.000000,80.000000,41.858370,-87.704655
50%,99.000000,95.000000,41.896740,-87.690298
75%,99.000000,99.000000,41.958017,-87.683179
max,99.000000,99.000000,42.009307,-87.664958


In [14]:
df_capstone2_.shape

(9, 10)

In [15]:
df_capstone2.describe()

,SAFETY_SCORE,Instruction Score,Latitude,Longitude
count,513.000000,513.000000,566.000000,566.000000
mean,49.504873,48.288499,41.841890,-87.677323
std,20.110837,17.417176,0.090361,0.058390
min,1.000000,1.000000,41.653674,-87.841052
25%,35.000000,37.000000,41.771866,-87.717152
50%,48.000000,47.000000,41.845521,-87.675878
75%,61.000000,59.000000,41.913584,-87.636059
max,99.000000,99.000000,42.021064,-87.533007


In [16]:
# the types of our dataset
df_capstone2.dtypes

NAME_OF_SCHOOL                         object
Street Address                         object
Elementary, Middle, or High School     object
SAFETY_SCORE                          float64
Instruction Score                     float64
Teachers Score                         object
Latitude                              float64
Longitude                             float64
COMMUNITY_AREA_NAME                    object
Location                               object
dtype: object

In [17]:
# we can see the mean
df_capstone2.mean()

SAFETY_SCORE         49.504873
Instruction Score    48.288499
Latitude             41.841890
Longitude           -87.677323
dtype: float64

In [18]:
df_capstone2.COMMUNITY_AREA_NAME = df_capstone2.COMMUNITY_AREA_NAME.str.lower()
df_capstone2

C:\Users\bootcamp\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,NAME_OF_SCHOOL,Street Address,"Elementary, Middle, or High School",SAFETY_SCORE,Instruction Score,Teachers Score,Latitude,Longitude,COMMUNITY_AREA_NAME,Location
0,Abraham Lincoln Elementary School,615 W Kemper Pl,ES,99.0,66.0,70,41.924497,-87.644522,lincoln park,"(41.92449696, -87.64452163)"
1,Adam Clayton Powell Paideia Community Academy ...,7511 S South Shore Dr,ES,54.0,84.0,76,41.760324,-87.556736,south shore,"(41.76032435, -87.55673627)"
2,Adlai E Stevenson Elementary School,8010 S Kostner Ave,ES,61.0,36.0,NDA,41.747111,-87.731702,ashburn,"(41.74711093, -87.73170248)"
3,Agustin Lara Elementary Academy,4619 S Wolcott Ave,ES,56.0,37.0,48,41.809757,-87.672145,new city,"(41.8097569, -87.6721446)"
4,Air Force Academy High School,3630 S Wells St,HS,49.0,55.0,54,41.828146,-87.632794,armour square,"(41.82814609, -87.63279369)"
...,...,...,...,...,...,...,...,...,...,...
561,William T Sherman Elementary School,1000 W 52nd St,ES,32.0,55.0,NDA,41.799788,-87.650255,new city,"(41.79978772, -87.65025483)"
562,William W Carter Elementary School,5740 S Michigan Ave,ES,13.0,35.0,62,41.789841,-87.622490,washington park,"(41.78984129, -87.62248974)"
563,Wolfgang A Mozart Elementary School,2200 N Hamlin Ave,ES,41.0,32.0,NDA,41.920927,-87.721925,logan square,"(41.92092734, -87.72192541)"
564,Woodlawn Community Elementary School,6657 S Kimbark Ave,ES,70.0,66.0,NDA,41.773400,-87.594356,woodlawn,"(41.77339962, -87.59435584)"


## Clustering

In [19]:
longitudes=[]
latitudes= []
neighs= []
for index in range(0,len(neig_list)):
    print(index)
    try:
        print(neig_list[index])
        query = neig_list[index]+ " Chicago,IL"
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(query)
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
        neighs.append(neig_list[index])
    except:

        latitudes.append(np.nan)
        longitudes.append(np.nan)
        neighs.append(neig_list[index])
        
df_3 =pd.DataFrame(dict(Neighborhoods=neighs,latitude=latitudes,longitude=longitudes))

0
Lower West Side
1
Avalon Park
2
Mckinley Park
3
Chicago Lawn
4
Fuller Park
5
Sauganash,Forest Glen
6
Dunning
7
Galewood
8
Printers Row
9
Jackson Park
10
Garfield Ridge
11
Oakland
12
Roseland
13
Hegewisch
14
Old Town
15
Grand Crossing
16
Morgan Park
17
Lincoln Park
18
Douglas
19
North Center
20
North Lawndale
21
Greektown
22
Edgewater
23
Chatham
24
Magnificent Mile
25
Humboldt Park
26
Wrigleyville
27
Grand Boulevard
28
Edison Park
29
Lincoln Square
30
Ukrainian Village
31
East Village
32
Millenium Park
33
Museum Campus
34
United Center
35
Near South Side
36
Belmont Cragin
37
Hyde Park
38
Uptown
39
New City
40
West Pullman
41
Bucktown
42
Armour Square
43
West Lawn
44
Woodlawn
45
Norwood Park
46
Andersonville
47
Garfield Park
48
Pullman
49
Mount Greenwood
50
Clearing
51
Jefferson Park
52
Englewood
53
South Deering
54
East Side
55
Washington Heights
56
Bridgeport
57
Streeterville
58
Portage Park
59
River North
60
O'Hare
61
Montclare
62
Grant Park
63
Gage Park
64
South Shore
65
Sheffield 

In [20]:
latitudes
longitudes

[-87.6656086,
 -87.5886584,
 -87.6736638,
 -87.6964408,
 -87.6325508,
 -87.73523436690189,
 -87.7964493,
 -87.7928372,
 -87.6288996,
 -87.63575316733329,
 -87.74548862491682,
 -87.6082424,
 -87.625881,
 -87.5469885,
 -87.6553579,
 -87.6003258,
 -87.6667156,
 -87.647832,
 -87.6179536,
 -87.6791596,
 -87.7183883,
 -87.6469318,
 -87.6639516,
 -87.6125481,
 -87.624228,
 -87.7041737,
 -87.6564773,
 -87.6172724,
 -87.81401633833357,
 -87.6896163305115,
 -87.6847499,
 -87.6244212,
 nan,
 120.9753251,
 -87.67418510441388,
 -87.6247738,
 -87.7686699,
 -87.5939244,
 -87.6555458,
 -87.6564403,
 -87.637823,
 -87.6701232,
 -87.633107,
 -87.722275,
 -87.599493,
 -87.80058173001102,
 -87.66927341557962,
 -87.71591710895399,
 -87.6066039,
 -87.7086616,
 -87.7733881,
 -87.7631179,
 -87.6458836,
 -87.57170331249044,
 -87.5333872,
 -87.65616,
 -87.651028,
 -87.6219967,
 -87.7650594,
 -87.8411818,
 -87.90676788547228,
 -87.8008931,
 -87.61928550074734,
 -87.6961636,
 -87.5752828,
 nan,
 -87.6244212,
 -87.

In [21]:
df_3

,Neighborhoods,latitude,longitude
0,Lower West Side,41.854200,-87.665609
1,Avalon Park,41.745035,-87.588658
2,Mckinley Park,41.831700,-87.673664
3,Chicago Lawn,41.775033,-87.696441
4,Fuller Park,41.818089,-87.632551
...,...,...,...
93,Little Village,41.875562,-87.624421
94,Avondale,41.938921,-87.711168
95,Calumet Heights,41.730035,-87.579213
96,Beverly,41.718153,-87.671767


In [22]:
#delete null values
df_3.dropna(inplace=True)

In [23]:
df_3.Neighborhoods =df_3.Neighborhoods.str.lower()
df_3.head()

,Neighborhoods,latitude,longitude
0,lower west side,41.854200,-87.665609
1,avalon park,41.745035,-87.588658
2,mckinley park,41.831700,-87.673664
3,chicago lawn,41.775033,-87.696441
4,fuller park,41.818089,-87.632551


## Foursquare

In [24]:
client_id = '0KTKKCYIO0BFEWJMDQ4ZTOADCGOED1KYXWIVS1JO3DPQ0LG1' # our Foursquare ID
client_secret = '5URQZPCQJT4U1ZOU1I1NX2AUYIWBFNVHC520MXQE3BFZRJYW' # our Foursquare Secret
version = '20180604'
radius = 3000
query= "Chicago,IL"                                                  



limit=500
categoryId ="4bf58dd8d48988d1f2931735"
url_root = " https://api.foursquare.com/v2/venues/search?"

In [25]:
# convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(query)
latitude = location.latitude
longitude = location.longitude
(latitude,longitude)

(41.8755616, -87.6244212)

In [26]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(client_id, client_secret, latitude, longitude, version, query, radius, limit,categoryId)
url

'https://api.foursquare.com/v2/venues/search?client_id=0KTKKCYIO0BFEWJMDQ4ZTOADCGOED1KYXWIVS1JO3DPQ0LG1&client_secret=5URQZPCQJT4U1ZOU1I1NX2AUYIWBFNVHC520MXQE3BFZRJYW&ll=41.8755616,-87.6244212&v=20180604&query=Chicago,IL&radius=3000&limit=500&categoryId=4bf58dd8d48988d1f2931735'

In [27]:
#show the dataset found in foursquare
results = requests.get(url).json()
results['response']['venues']

[{'id': '5a7b7784b9ac383f9f6621d6',
  'name': 'American Rhythm Center, 410 S Michigan Ave, 3Rd Floor, Chicago, Il',
  'location': {'address': '410 S Michigan Ave',
   'lat': 41.876447999999996,
   'lng': -87.62485699999999,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.876447999999996,
     'lng': -87.62485699999999}],
   'distance': 105,
   'cc': 'US',
   'city': 'Chicago',
   'state': 'IL',
   'country': 'United States',
   'formattedAddress': ['410 S Michigan Ave', 'Chicago, IL', 'United States']},
  'categories': [{'id': '4bf58dd8d48988d134941735',
    'name': 'Dance Studio',
    'pluralName': 'Dance Studios',
    'shortName': 'Dance Studio',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_dancestudio_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1595106346',
  'hasPerk': False},
 {'id': '4ae786a5f964a52019ac21e3',
  'name': 'Auditorium Theatre',
  'location': {'address': '50 E Congress Pkwy',
   'c

In [28]:
#let's see our venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,venuePage.id,location.neighborhood
0,5a7b7784b9ac383f9f6621d6,"American Rhythm Center, 410 S Michigan Ave, 3R...","[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",v-1595106346,False,410 S Michigan Ave,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S Michigan Ave, Chicago, IL, United States]",NaN,NaN,NaN,NaN
1,4ae786a5f964a52019ac21e3,Auditorium Theatre,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",v-1595106346,False,50 E Congress Pkwy,41.876058,-87.625303,"[{'label': 'display', 'lat': 41.87605807115569...",91,US,Chicago,IL,United States,"[50 E Congress Pkwy (Roosevelt University), Ch...",Roosevelt University,60605,33207795,NaN
2,4c45c0e0dcd61b8d961f7d56,Chicago Youth Symphony Orchestras,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",v-1595106346,False,410 S Michigan Ave Ste 833,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S Michigan Ave Ste 833 (Van Buren), Chica...",Van Buren,60605,NaN,NaN
3,50175d13e4b002d58e736599,Chicago Human Rhythm Project (@ The Fine Arts ...,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",v-1595106346,False,410 S Michigan Avenue,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S Michigan Avenue, Chicago, IL 60605, Uni...",NaN,60605,NaN,NaN
4,50177c8de4b0d51f8e12ca0a,Chicago Human Rhythm Project - American Rhythm...,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",v-1595106346,False,410 S. Michigan Avenue,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S. Michigan Avenue, Chicago, IL 60605, Un...",NaN,60605,NaN,NaN
5,4aa05f40f964a520643f20e3,The Chicago Theatre,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",v-1595106346,False,175 N State St,41.885539,-87.627151,"[{'label': 'display', 'lat': 41.8855389313031,...",1133,US,Chicago,IL,United States,"[175 N State St (at E Lake St), Chicago, IL 60...",at E Lake St,60601,35000930,NaN
6,4dbb90fcbe4d6d4c08fc896f,Chicago Jazz Ensemble,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",v-1595106346,False,600 S Michigan Ave,41.874433,-87.624252,"[{'label': 'display', 'lat': 41.874433, 'lng':...",126,US,Chicago,IL,United States,"[600 S Michigan Ave, Chicago, IL 60605, United...",NaN,60605,NaN,NaN
7,4adfca6df964a520777d21e3,Symphony Center (Chicago Symphony Orchestra),"[{'id': '5032792091d4c4b30a586d5c', 'name': 'C...",v-1595106346,False,220 S Michigan Ave,41.879275,-87.624680,"[{'label': 'display', 'lat': 41.87927535633318...",413,US,Chicago,IL,United States,"[220 S Michigan Ave (btwn Adams & Jackson), Ch...",btwn Adams & Jackson,60604,NaN,"Chicago Loop, IL"
8,5036ca0be4b0e82a0177daf0,Chicago Summer Dance in Grant Park,"[{'id': '4bf58dd8d48988d1f2931735', 'name': 'P...",v-1595106346,False,NaN,41.874163,-87.623793,"[{'label': 'display', 'lat': 41.87416283696295...",164,US,Chicago,IL,United States,"[Chicago, IL 60605, United States]",NaN,60605,NaN,NaN
9,4a97023cf964a520f32720e3,Chicago Shakespeare Theater,"[{'id': '4bf58dd8d48988d137941735', 'name': 'T...",v-1595106346,False,800 E Grand Ave,41.891299,-87.606093,"[{'label': 'display', 'lat': 41.89129883721647...",2318,US,Chicago,IL,United States,"[800 E Grand Ave (at Navy Pier), Chicago, IL 6...",at Navy Pier,60611,66613895,NaN


In [29]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,"American Rhythm Center, 410 S Michigan Ave, 3R...",Dance Studio,410 S Michigan Ave,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S Michigan Ave, Chicago, IL, United States]",NaN,NaN,NaN,5a7b7784b9ac383f9f6621d6
1,Auditorium Theatre,Theater,50 E Congress Pkwy,41.876058,-87.625303,"[{'label': 'display', 'lat': 41.87605807115569...",91,US,Chicago,IL,United States,"[50 E Congress Pkwy (Roosevelt University), Ch...",Roosevelt University,60605,NaN,4ae786a5f964a52019ac21e3
2,Chicago Youth Symphony Orchestras,Performing Arts Venue,410 S Michigan Ave Ste 833,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S Michigan Ave Ste 833 (Van Buren), Chica...",Van Buren,60605,NaN,4c45c0e0dcd61b8d961f7d56
3,Chicago Human Rhythm Project (@ The Fine Arts ...,Dance Studio,410 S Michigan Avenue,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S Michigan Avenue, Chicago, IL 60605, Uni...",NaN,60605,NaN,50175d13e4b002d58e736599
4,Chicago Human Rhythm Project - American Rhythm...,Dance Studio,410 S. Michigan Avenue,41.876448,-87.624857,"[{'label': 'display', 'lat': 41.87644799999999...",105,US,Chicago,IL,United States,"[410 S. Michigan Avenue, Chicago, IL 60605, Un...",NaN,60605,NaN,50177c8de4b0d51f8e12ca0a
5,The Chicago Theatre,Theater,175 N State St,41.885539,-87.627151,"[{'label': 'display', 'lat': 41.8855389313031,...",1133,US,Chicago,IL,United States,"[175 N State St (at E Lake St), Chicago, IL 60...",at E Lake St,60601,NaN,4aa05f40f964a520643f20e3
6,Chicago Jazz Ensemble,Performing Arts Venue,600 S Michigan Ave,41.874433,-87.624252,"[{'label': 'display', 'lat': 41.874433, 'lng':...",126,US,Chicago,IL,United States,"[600 S Michigan Ave, Chicago, IL 60605, United...",NaN,60605,NaN,4dbb90fcbe4d6d4c08fc896f
7,Symphony Center (Chicago Symphony Orchestra),Concert Hall,220 S Michigan Ave,41.879275,-87.624680,"[{'label': 'display', 'lat': 41.87927535633318...",413,US,Chicago,IL,United States,"[220 S Michigan Ave (btwn Adams & Jackson), Ch...",btwn Adams & Jackson,60604,"Chicago Loop, IL",4adfca6df964a520777d21e3
8,Chicago Summer Dance in Grant Park,Performing Arts Venue,NaN,41.874163,-87.623793,"[{'label': 'display', 'lat': 41.87416283696295...",164,US,Chicago,IL,United States,"[Chicago, IL 60605, United States]",NaN,60605,NaN,5036ca0be4b0e82a0177daf0
9,Chicago Shakespeare Theater,Theater,800 E Grand Ave,41.891299,-87.606093,"[{'label': 'display', 'lat': 41.89129883721647...",2318,US,Chicago,IL,United States,"[800 E Grand Ave (at Navy Pier), Chicago, IL 6...",at Navy Pier,60611,NaN,4a97023cf964a520f32720e3


In [30]:
# here we filtre our columns ['name','categories','address','lat','lng']
df_capstone3 = dataframe_filtered[['name','categories','address','lat','lng','neighborhood']]
df_capstone3.head()

,name,categories,address,lat,lng,neighborhood
0,"American Rhythm Center, 410 S Michigan Ave, 3R...",Dance Studio,410 S Michigan Ave,41.876448,-87.624857,NaN
1,Auditorium Theatre,Theater,50 E Congress Pkwy,41.876058,-87.625303,NaN
2,Chicago Youth Symphony Orchestras,Performing Arts Venue,410 S Michigan Ave Ste 833,41.876448,-87.624857,NaN
3,Chicago Human Rhythm Project (@ The Fine Arts ...,Dance Studio,410 S Michigan Avenue,41.876448,-87.624857,NaN
4,Chicago Human Rhythm Project - American Rhythm...,Dance Studio,410 S. Michigan Avenue,41.876448,-87.624857,NaN


# Here the maps of all  Elementary School at Chicago

In [31]:
address = 'Chicago,IL'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Chicago,IL are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Chicago,IL are 41.8755616, -87.6244212.


In [32]:
 #create map of CHicago using latitude and longitude values
chicago_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for NAME_OF_SCHOOL, lat, lng,COMMUNITY_AREA_NAME in zip(df_capstone2_['NAME_OF_SCHOOL'], df_capstone2_['Latitude'], df_capstone2_['Longitude'],df_capstone2_['COMMUNITY_AREA_NAME']):
    label = '{},{}'.format(NAME_OF_SCHOOL,COMMUNITY_AREA_NAME)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(chicago_map)  
    
chicago_map

In [33]:
df_capstone3 = dataframe_filtered[['name','categories','address','lat','lng']]
df_capstone3

,name,categories,address,lat,lng
0,"American Rhythm Center, 410 S Michigan Ave, 3R...",Dance Studio,410 S Michigan Ave,41.876448,-87.624857
1,Auditorium Theatre,Theater,50 E Congress Pkwy,41.876058,-87.625303
2,Chicago Youth Symphony Orchestras,Performing Arts Venue,410 S Michigan Ave Ste 833,41.876448,-87.624857
3,Chicago Human Rhythm Project (@ The Fine Arts ...,Dance Studio,410 S Michigan Avenue,41.876448,-87.624857
4,Chicago Human Rhythm Project - American Rhythm...,Dance Studio,410 S. Michigan Avenue,41.876448,-87.624857
5,The Chicago Theatre,Theater,175 N State St,41.885539,-87.627151
6,Chicago Jazz Ensemble,Performing Arts Venue,600 S Michigan Ave,41.874433,-87.624252
7,Symphony Center (Chicago Symphony Orchestra),Concert Hall,220 S Michigan Ave,41.879275,-87.624680
8,Chicago Summer Dance in Grant Park,Performing Arts Venue,NaN,41.874163,-87.623793
9,Chicago Shakespeare Theater,Theater,800 E Grand Ave,41.891299,-87.606093


In [34]:
df_capstone4 = df_capstone3[df_capstone3["categories"] == 'Dance Studio']

In [41]:
df_capstone4

,name,categories,address,lat,lng
0,"American Rhythm Center, 410 S Michigan Ave, 3R...",Dance Studio,410 S Michigan Ave,41.876448,-87.624857
3,Chicago Human Rhythm Project (@ The Fine Arts ...,Dance Studio,410 S Michigan Avenue,41.876448,-87.624857
4,Chicago Human Rhythm Project - American Rhythm...,Dance Studio,410 S. Michigan Avenue,41.876448,-87.624857
11,Chicago Multicultural Dance Center,Dance Studio,806 S Plymouth Ct,41.871843,-87.629103
14,Chicago Multicultural Dance Center,Dance Studio,47 W Polk St,41.871843,-87.629103
25,Chicago Dance Academy,Dance Studio,1920 S Wabash Ave,41.855942,-87.626069
26,Chicago Lindy Hop,Dance Studio,415 W Huron St,41.894485,-87.639174
27,Chicago Dance,Dance Studio,"731 N Sangamon St, 3rd Floor",41.894675,-87.638915
28,Chicago Dance,Dance Studio,415 W Huron St,41.894499,-87.639189
34,SummerDance,Dance Studio,601 S Michigan Ave,41.873804,-87.623335


In [36]:
chicago_map2 = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for name, lat, lng in zip(df_capstone4['name'], df_capstone4['lat'], df_capstone4['lng']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(chicago_map2)  
    
chicago_map2

In [37]:
df_capstone4.shape

(12, 5)

In [38]:
df_capstone2_.rename(columns={"Street Address": "address","Elementary, Middle, or High School":"Grade","COMMUNITY_AREA_NAME":"borough"}, errors="raise")

,NAME_OF_SCHOOL,address,Grade,SAFETY_SCORE,Instruction Score,Teachers Score,Latitude,Longitude,borough,Location
27,Annie Keller Elementary Gifted Magnet School,3020 W 108th St,ES,99.0,82.0,82,41.697198,-87.697264,MOUNT GREENWOOD,"(41.69719792, -87.6972638)"
141,Ellen Mitchell Elementary School,2233 W Ohio St,ES,99.0,95.0,90,41.892055,-87.683179,WEST TOWN,"(41.89205482, -87.68317867)"
174,Frederic Chopin Elementary School,2450 W Rice St,ES,86.0,99.0,99,41.896740,-87.688902,WEST TOWN,"(41.89674048, -87.68890154)"
245,James G Blaine Elementary School,1420 W Grace St,ES,99.0,74.0,NDA,41.950808,-87.664958,LAKE VIEW,"(41.95080812, -87.66495825)"
366,Mary E Courtenay Elementary Language Arts Center,1726 W Berteau Ave,ES,99.0,80.0,NDA,41.958017,-87.671893,LAKE VIEW,"(41.9580169, -87.6718933)"
417,Oriole Park Elementary School,5424 N Oketo Ave,ES,99.0,71.0,NDA,41.978515,-87.812091,NORWOOD PARK,"(41.97851501, -87.81209057)"
488,Stephen Decatur Classical Elementary School,7030 N Sacramento Ave,ES,99.0,99.0,99,42.009307,-87.704655,WEST RIDGE,"(42.00930696, -87.70465502)"
494,Talman Elementary School,5450 S Talman Ave,ES,99.0,99.0,81,41.794074,-87.690298,GAGE PARK,"(41.79407382, -87.6902983)"
555,William Penn Elementary School,1616 S Avers Ave,ES,78.0,99.0,NDA,41.858370,-87.721336,NORTH LAWNDALE,"(41.85836995, -87.72133609)"


In [39]:
df_capstone2_['NAME_OF_SCHOOL'].head()

27         Annie Keller Elementary Gifted Magnet School
141                    Ellen Mitchell Elementary School
174                   Frederic Chopin Elementary School
245                    James G Blaine Elementary School
366    Mary E Courtenay Elementary Language Arts Center
Name: NAME_OF_SCHOOL, dtype: object

In [40]:
chicago_map3 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for NAME_OF_SCHOOL, lat, lng in zip(df_capstone2_['NAME_OF_SCHOOL'], df_capstone2_['Latitude'], df_capstone2_['Longitude']):
    label = '{}'.format(NAME_OF_SCHOOL)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(chicago_map3)
    
for name, lat2, lng2 in zip (df_capstone4['name'], df_capstone4['lat'], df_capstone4['lng']):
     label1 = '{}'.format(name)                                                          
     label1 = folium.Popup(label1, parse_html=True)                                                           
     folium.CircleMarker(
        [lat2, lng2],
        radius=4,
        popup=label1,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(chicago_map3) 
chicago_map3

# Results  <a name="results"></a>

There are many elementary schools in Chicago about 566 elementary schools in various area communities, A GPS could guide you well or a map.

Our problem was to find a Dance School close to an Elementary School at Chicago and that the elementary school would be a good school by certain criteria such as:

   * SAFETY_SCORE
   * Instruction Score
   * Teachers Score


Enough security of very competent teachers and a very high level of instruction so that the children would be literally well 


After doing our analysis and research through Foursquare, Folium, Clustering and others, there are two Elementary schools with the required criteria that are located near a dance school in Chicago  their names are:
   * Michell Elementary School
   * Chopin Elementary School




# Conclusion <a name="conclusion"></a>

Parents will be able to manage their time when they put their children in an elementary school with a high education score and find a **Dance School** close enough to that **Elementary School** and its **Elementary School** with high education score is good enough to take care of the students' education. Even if there are parents who will not put their children in a ready-made dance school.